## AI Road Safety Project

### 5. Deployment(배포)
개발 제품 서비스(Gradio, Flask, Streamit 등 활용) 플랫폼에서 테스트

#### 5.1 배포를 위해 필요한 라이브러리 불러오기
개발된 AI Project을 어떤 형태로 서비스 할 것인지를 결정하고 필요한 라이브러리를 불러온다. 여기서는 Gradio를 활용한 것으로 한다.

In [1]:
#필요 라이브러리 불러오기

import openvino as ov
import gradio as gr
import yaml
import numpy as np
import cv2

#### 5.2 개발한 모델 불러오기
모델 평가를 통해 선정한 최적 모델을 불러온다. 모델은 models 폴더에 저장한다.

In [2]:
# 모델 불러오기
core = ov.Core()

model = core.read_model(model="models/best.xml")
compiled_model = core.compile_model(model=model, device_name="CPU")

input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

print("Input layer shape: ", input_layer.shape)
print("Output layer shape:", output_layer.shape)

Input layer shape:  [1,3,640,640]
Output layer shape: [1,5,8400]


In [3]:
# 레이블 불러오기
with open('models/metadata.yaml') as info:
      info_dict = yaml.load(info, Loader=yaml.Loader)
        
labels = info_dict['names']
print(labels)

{0: 'pothole'}


#### 5.3 새로운 입력 데이터 Shape 맞추기
새로 입력될 데이터의 Shape를 맞추어 주는 함수를 만든다.

In [4]:
# 새로운 데이터 입력 shape 맞추는 함수
def prepare_data(image):
    input_w, input_h = 640, 640
    input_image = cv2.resize(image, (input_w,input_h))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = input_image/255

    input_image = input_image.transpose(2, 0, 1)
    input_image = np.expand_dims(input_image, 0)
    
    return input_image

#### 5.4 추론 결과 저장하기
새로 입력된 데이터의 추론 한 결과 값을 얻는다.

In [14]:
# 새로운 이미지 데이터 추론 함수
def predict_image(image, conf_threshold):  
    input_image = prepare_data(image)
    output = compiled_model([input_image])[output_layer]
    boxes, scores, label_key = evaluate(output, conf_threshold)
    if len(boxes):
        nms_output = non_max_suppression(boxes, scores, conf_threshold)
        visualize(nms_output, boxes, image, label_key, scores, colors_dict)
    return image

In [15]:
# 새로운 영상 데이터 추론 함수
def predict_video(vid_input, conf_threshold): 
    colors_dict = create_colors(labels)
    video_capture = cv2.VideoCapture(vid_input)
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    fourcc = cv2.VideoWriter_fourcc(*'vp80')
    vid_name= 'output.webm'
    width_out  = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height_out = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(vid_name, fourcc, 20.0, (width_out, height_out))
    while(True):
        ret, frame = video_capture.read()
        if frame is None:
            break
        input_image = prepare_data(frame)
        output = compiled_model([input_image])[output_layer]        
        boxes, scores, label_names = evaluate(output, conf_threshold)
 
        if len(boxes):
            nms_output = non_max_suppression(boxes, scores, .5)
            visualize(nms_output, boxes, frame, label_names, scores, colors_dict)

        out.write(frame)
        
    out.release()
    file_path = "output.webm"
    return file_path

#### 5.5 추론 결과 후처리: Post processing with the result of inference
- confidence 이상의 바운딩 박스와 레이블 찾기

In [5]:
# confidence 이상의 바운딩 박스 찾는 함수
def evaluate(output, conf):   
    boxes = []
    scores = []
    label_key = []
    label_index = 0
    
    for class_ in output[0][4:]:      
        for index in range (len(class_)):
            confidence = class_[index]
            
            if  confidence > conf:
                xcen = output[0][0][index]
                ycen = output[0][1][index]
                w = output[0][2][index]
                h = output[0][3][index]

                xmin = int(xcen - (w/2))
                xmax = int(xcen + (w/2))
                ymin = int(ycen - (h/2))
                ymax = int(ycen + (h/2))

                box = (xmin, ymin, xmax, ymax)
                boxes.append(box)
                scores.append(confidence)

                label_key.append(label_index)
  
        label_index += 1 
        
    boxes = np.array(boxes)
    scores = np.array(scores)
    
    return boxes, scores, label_key

- 박스 하나 찾기

In [16]:
# 가장 큰 
def non_max_suppression(boxes, scores, iou_threshold):	
    assert boxes.shape[0] == scores.shape[0]
    ys1 = boxes[:, 0]
    xs1 = boxes[:, 1]
    ys2 = boxes[:, 2]
    xs2 = boxes[:, 3]
    areas = (ys2 - ys1) * (xs2 - xs1)
  
    scores_indexes = scores.argsort().tolist()   
    boxes_keep_index = []
    
    while len(scores_indexes):
        index = scores_indexes.pop()
        boxes_keep_index.append(index)  
        if not len(scores_indexes):
            break
        ious = compute_iou(boxes[index], boxes[scores_indexes], areas[index], areas[scores_indexes])
        filtered_indexes = set((ious > iou_threshold).nonzero()[0])
        scores_indexes = [
            v for (i, v) in enumerate(scores_indexes)
            if i not in filtered_indexes
        ]
    return np.array(boxes_keep_index)
    
def compute_iou(box, boxes, box_area, boxes_area):
    assert boxes.shape[0] == boxes_area.shape[0]
    ys1 = np.maximum(box[0], boxes[:, 0])
    xs1 = np.maximum(box[1], boxes[:, 1])
    ys2 = np.minimum(box[2], boxes[:, 2])
    xs2 = np.minimum(box[3], boxes[:, 3])

    intersections = np.maximum(ys2 - ys1, 0) * np.maximum(xs2 - xs1, 0)
    unions = box_area + boxes_area - intersections
    ious = intersections / unions
    return ious

- 레이블별 박스 컬러 정하기

In [17]:
# 박스 컬러 함수
def create_colors(labels):
    colors_dict = {}
    for i in range(len(labels)):
        random_color = list(np.random.randint(0,255, size =3))
        random_color = ( int (random_color [ 0 ]), int (random_color [ 1 ]), int (random_color [ 2 ])) 
        colors_dict[i] = random_color
    return colors_dict

- 박스 그리기 및 레이블 출력하기

In [9]:
# 시각화 함수
def visualize(nms_output, boxes, orig_image, label_key,scores, colors_dict ):
    orig_h, orig_w, c = orig_image.shape 
    for i in nms_output:
        xmin, ymin, xmax, ymax = boxes[i]
        xmin = int(xmin*orig_w/640)
        ymin = int(ymin*orig_h/640)
        xmax = int(xmax*orig_w/640)
        ymax = int(ymax*orig_h/640)
        color = colors_dict[label_key[i]]
        cv2.rectangle(orig_image, (xmin,ymin), (xmax,ymax), color, 4)   
        text = str(int(np.rint(scores[i]*100))) + "% " + str(labels[label_key[i]])
        cv2.putText(orig_image, text, (xmin+2,ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0, 0, 0), 1, cv2.LINE_AA)

#### 5.6 Web Deployment with Gradio
새로 입력될 데이터의 Shape를 맞추어 주는 함수를 만든다.

In [12]:
image_iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Slider(minimum=0, maximum=1, value=0.25, label="Confidence threshold"),
    ],
    outputs=gr.Image(label="Result"),
)

video_iface = gr.Interface(
    fn=predict_video,
    inputs=[
        gr.Video(label = "Upload Video"), 
        gr.Slider(minimum=0, maximum=1, value=0.25, label="Confidence threshold"),
    ],
    outputs=gr.Video(label="Result"),
)

demo = gr.TabbedInterface([image_iface, video_iface], ["Image", "Video"])

In [13]:
colors_dict = create_colors(labels)

if __name__ == '__main__':
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
